In [1]:
print('hi')

hi


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


C:\Users\kingr\AppData\Roaming\Python\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pinecone_API='d73c6e08-3a17-47c7-bcb6-57e1802e0c7b'
pinecone_API_env='gcp-starter'

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data")

In [6]:
# !pip install pypdf

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [12]:
query_result = embeddings.embed_query("He")
print("Length", len(query_result))

Length 384


In [13]:
query_result

[-0.10931986570358276,
 0.10040654987096786,
 -0.005641328636556864,
 0.0011855927295982838,
 -0.0045854742638766766,
 -0.06776651740074158,
 0.14050930738449097,
 -0.008834164589643478,
 -0.04340279847383499,
 -0.038659099489450455,
 -0.07604701071977615,
 -0.04677673429250717,
 0.01155656948685646,
 0.050692368298769,
 0.09095363318920135,
 -0.009140732698142529,
 -0.05636376515030861,
 0.037493571639060974,
 -0.09618627279996872,
 -0.01834997348487377,
 -0.060110557824373245,
 0.10646678507328033,
 -0.007165154907852411,
 -0.018598323687911034,
 -0.01830114610493183,
 -0.028916604816913605,
 0.058159615844488144,
 -0.006956805475056171,
 0.06762663275003433,
 -0.06134425848722458,
 0.03938421979546547,
 -0.04704694077372551,
 0.08458548039197922,
 0.02119159698486328,
 -0.0017544409492984414,
 -0.055822793394327164,
 -0.0070959534496068954,
 -0.03404630348086357,
 -0.006312417332082987,
 0.0379212461411953,
 -0.00393513310700655,
 -0.06869405508041382,
 -0.025054767727851868,
 0.061

In [14]:
# # Initialize Pinecone with API key and environment
# pc = Pinecone(api_key=pinecone_API, environment=pinecone_API_env)

# index_name = "medical-chatbot"

# # Create the index if it doesn't exist
# if index_name not in pc.list_indexes().names():
#     pc.create_index(
#         name=index_name,
#         dimension=embeddings.dimension,  # Ensure dimension matches embeddings
#         metric='cosine',  # Common metric for text embeddings
#     )

# # Create embeddings and store in the index
# docsearch = pc.upsert(
#     documents=[
#         {"id": str(i), "text": t.page_content, "embedding": embeddings[t.page_content]}
#         for i, t in enumerate(text_chunks)
#     ],
#     index_name=index_name,
# )

In [15]:
def simple(x):
    print(x)
    return str(x)

In [16]:

pc = Pinecone(api_key=pinecone_API)
index = pc.Index("medical-chatbot")

# index.upsert(
#   vectors=[
#         {"id": simple(i),  "values": embeddings.embed_query(t.page_content)}
#         for i, t in enumerate(text_chunks[0:999])
#     ],
# )

In [17]:
def retrive(query):
  res=index.query(vector=embeddings.embed_query(str(query)),  top_k=3,
    include_values=True)
  return [text_chunks[int(x["id"])] for x in res['matches']]

In [18]:
retrive('talk about Accessory organ')

[Document(page_content='KEY TERMS\nAccessory organ —A lump of tissue adjacent to an\norgan that is similar to it, but which serves noimportant purpose, if functional at all. While notnecessarily harmful, such organs can cause prob-lems if they grow too large or become cancerous. Inany case, their presence points to an underlyingabnormality in the parent organ.\nBenign —In medical usage, benign is the opposite of\nmalignant. It describes an abnormal growth that isstable, treatable and generally not life-threatening.', metadata={'source': 'data\\Medical_book.pdf', 'page': 16}),
 Document(page_content='which organs and areas are', metadata={'source': 'data\\Medical_book.pdf', 'page': 55}),
 Document(page_content='THE ORGAN SYSTEM. In the Chinese system, there', metadata={'source': 'data\\Medical_book.pdf', 'page': 55})]

In [19]:
text_chunks[981]

Document(page_content='tion (New Y ork). Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 83', metadata={'source': 'data\\Medical_book.pdf', 'page': 96})

In [20]:
embeddings.embed_query(text_chunks[5].page_content)

[-0.005056857597082853,
 0.04913092404603958,
 -0.04827364906668663,
 0.0733647271990776,
 0.004527191631495953,
 0.012238346971571445,
 -0.026310419663786888,
 -0.005213056690990925,
 0.059581100940704346,
 -0.0293669942766428,
 0.10747122019529343,
 -0.040049660950899124,
 0.05020938068628311,
 -0.05857820063829422,
 -0.05427786335349083,
 -0.04137570783495903,
 -0.005753800272941589,
 0.031099259853363037,
 -0.03989054262638092,
 0.0060156467370688915,
 -0.02551540732383728,
 -0.015900876373052597,
 -0.07473645359277725,
 0.023733124136924744,
 0.0681617259979248,
 -0.018373003229498863,
 -0.13871367275714874,
 -0.04481825232505798,
 0.05499320849776268,
 -0.09119535982608795,
 0.06329549849033356,
 0.12256187200546265,
 0.0898047462105751,
 0.05252948775887489,
 0.07832647860050201,
 0.01869102567434311,
 -0.01391858421266079,
 -0.03642899915575981,
 -0.09474660456180573,
 0.0533127561211586,
 0.02362591028213501,
 0.004525855183601379,
 -0.026432717218995094,
 -0.05702587962150574

In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
llm=CTransformers(model="../llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [24]:
# from langchain.llms import LLM
# from langchain.vectorstores import BaseVectorStore
# from langchain.chains.retrieval_qa import BaseRetriever

# class CustomRetriever(BaseRetriever):
#     def __init__(self, embeddings: LLM, index: BaseVectorStore):
#         self.embeddings = embeddings
#         self.index = index

#     def _aget_relevant_documents(self, query: str) -> List[Document]:
#         return retrive(query)  # Call your function here


In [25]:
# !pip install --upgrade langchain

In [26]:
# qa=RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=retrive('white blood'),
#     return_source_documents=True, 
#     chain_type_kwargs=chain_type_kwargs)

In [27]:
# from langchain_community.chat_models import ChatOpenAI
# from langchain_core.documents import Document
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain


In [28]:
!python --version

Python 3.11.1


In [30]:
# import langchain
# from langchain.llms import CTransformers
# from langchain.agents import RetrievalChatAgent
# from langchain.prompts import PromptTemplate
# from langchain.memory import ConversationBufferMemory

# # Load your LLM
# llm = CTransformers(
#     model="../llama-2-7b-chat.ggmlv3.q4_0.bin",
#     model_type="llama",
#     config={'max_new_tokens': 512, 'temperature': 0.8}
# )

# # Define the prompt template
# PROMPT = PromptTemplate(
#     template=prompt_template,
#     input_variables=["context", "question"]
# )

# # Create the RetrievalChatAgent
# agent = RetrievalChatAgent(
#     llm=llm,
#     prompt_template=PROMPT,
#     memory=ConversationBufferMemory(max_turns=3)  # Adjust as needed
# )

# # ... (rest of the code from previous response)


In [4]:
!pip show langchain

Name: langchain
Version: 0.1.5
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\kingr\AppData\Roaming\Python\Python311\site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [29]:
def handle_user_query(user_query):
    retrieved_documents = retrive(user_query)

    # Combine retrieved document contents into a single string
    retrieved_text = "\n".join([doc.page_content for doc in retrieved_documents])

    
    return retrieved_text


In [32]:
query = input("You: ")
print('you: ',query)
prompt = prompt_template.format(context=handle_user_query(query),question=query)
print('prompt: ',prompt)
response = llm.invoke(prompt)
print("llm:", response)


you:  talk about Accessory organ
prompt:  
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: KEY TERMS
Accessory organ —A lump of tissue adjacent to an
organ that is similar to it, but which serves noimportant purpose, if functional at all. While notnecessarily harmful, such organs can cause prob-lems if they grow too large or become cancerous. Inany case, their presence points to an underlyingabnormality in the parent organ.
Benign —In medical usage, benign is the opposite of
malignant. It describes an abnormal growth that isstable, treatable and generally not life-threatening.
which organs and areas are
THE ORGAN SYSTEM. In the Chinese system, there
Question: talk about Accessory organ

Only return the helpful answer below and nothing else.
Helpful answer:

llm: The accessory organ in the human body is the appendix.


In [34]:
llm.invoke('who are you')

'?\n Einzelnen. What do you want to know?  I am an AI designed by Meta AI to generate creative content, such as stories, poems, and dialogue. I can understand and respond to questions and prompts in a conversational manner. Would you like me to create something for you?'